# Early Church Martyrdoms (1st-5th Centuries)

This notebook analyzes recorded martyrdoms in the early Christian church across the first five centuries.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

## Data Collection

Create a comprehensive dataset of recorded martyrdoms from the first 5 centuries of the church.

In [ ]:
# Create the martyrdoms dataset
martyrdoms_data = {
    'Year': [34, 43, 54, 60, 64, 68, 72, 81, 95, 107, 155, 165, 177, 203, 250, 258, 303, 313, 335, 451],
    'Deaths': [1, 1, 1, 1, 100, 1, 3, 100, 1, 1, 1, 7, 48, 5, 100, 3, 1000, 0, 100, 100],
    'Emperor': ['Tiberius', 'Claudius', 'Nero', 'Nero', 'Nero', 'Nero', 'Vespasian', 'Domitian', 'Domitian', 'Trajan', 
                'Antoninus Pius', 'Marcus Aurelius', 'Marcus Aurelius', 'Septimius Severus', 'Decius', 'Valerian', 
                'Diocletian', 'Constantine', 'Constantine II', 'Theodosius II'],
    'Event': ['Stephen', 'James the Greater', 'Philip the Apostle', 'Matthew the Apostle', 
              "Nero's Persecution", 'Mark the Apostle', 'Jude, Simon, Thomas (apostles)', 
              "Domitian's Persecution", 'John (exile)', 'Ignatius of Antioch', 'Polycarp', 
              'Justin Martyr and companions', 'Lyon Persecution', 'Perpetua & Felicity', 
              'Decius Persecution', 'Sixtus II & Lawrence', 'Diocletian Great Persecution', 
              'Edict of Milan (end of persecution)', "Shapur II's Persecution", 'Battle of Avarayr (Armenia)'],
    'Location': ['Jerusalem', 'Jerusalem', 'Hieropolis', 'Ethiopia', 'Rome', 'Alexandria', 
                 'Jerusalem/Edessa/India', 'Roman Empire', 'Patmos/Ephesus', 'Rome', 'Smyrna', 
                 'Rome', 'Lyon (Gaul)', 'Carthage', 'Roman Empire', 'Rome/Carthage', 
                 'Roman Empire', 'Roman Empire', 'Persian Empire', 'Armenia']
}

df = pd.DataFrame(martyrdoms_data)
df['Period'] = pd.cut(df['Year'], bins=[0, 100, 200, 300, 400, 500], 
                       labels=['1st Century', '2nd Century', '3rd Century', '4th Century', '5th Century'])

print(df.to_string(index=False))
print(f"\nTotal estimated martyrs: {df['Deaths'].sum():,}")

## Summary Statistics by Century

In [ ]:
# Summary statistics by century
century_summary = df.groupby('Period').agg({
    'Deaths': ['sum', 'mean', 'count'],
    'Year': ['min', 'max']
}).round(2)

print("\nMartydoms by Century:")
print(century_summary)

## Interactive Visualization: Martyrdoms Over Time

This interactive chart shows the estimated number of deaths from recorded Christian martyrdoms across five centuries.

In [ ]:
# Create interactive Plotly bar chart
fig = go.Figure()

# Color code by century
colors = {
    '1st Century': '#0ea5e9',
    '2nd Century': '#06b6d4',
    '3rd Century': '#f97316',
    '4th Century': '#ef4444',
    '5th Century': '#8b5cf6'
}

for period in df['Period'].unique():
    period_data = df[df['Period'] == period]
    fig.add_trace(go.Bar(
        x=period_data['Year'],
        y=period_data['Deaths'],
        name=period,
        marker_color=colors[period],
        hovertemplate='<b>%{customdata[0]}</b><br>' +
                      'Year: %{x} AD<br>' +
                      'Deaths: %{y}<br>' +
                      'Emperor: %{customdata[1]}<br>' +
                      'Location: %{customdata[2]}<extra></extra>',
        customdata=period_data[['Event', 'Emperor', 'Location']].values
    ))

fig.update_layout(
    title={
        'text': 'Early Church Martyrdoms (1st-5th Centuries)',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'color': '#1e293b'}
    },
    xaxis_title='Year (AD)',
    yaxis_title='Estimated Number of Deaths',
    hovermode='x unified',
    template='plotly_white',
    height=600,
    font=dict(size=12),
    xaxis=dict(
        showgrid=True,
        gridwidth=1,
        gridcolor='#e2e8f0'
    ),
    yaxis=dict(
        showgrid=True,
        gridwidth=1,
        gridcolor='#e2e8f0'
    ),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)

fig.show()

# Save for Myst integration
fig.write_html('martyrdoms_chart.html')
print("Chart saved as 'martyrdoms_chart.html'")

## Cumulative Martyrdoms Over Time

In [ ]:
# Create cumulative deaths visualization
df_sorted = df.sort_values('Year')
df_sorted['Cumulative_Deaths'] = df_sorted['Deaths'].cumsum()

fig_cumulative = go.Figure()

fig_cumulative.add_trace(go.Scatter(
    x=df_sorted['Year'],
    y=df_sorted['Cumulative_Deaths'],
    mode='lines+markers',
    name='Cumulative Deaths',
    line=dict(color='#ef4444', width=3),
    marker=dict(size=8),
    hovertemplate='<b>Year %{x} AD</b><br>' +
                  'Cumulative Deaths: %{y}<extra></extra>'
))

fig_cumulative.update_layout(
    title={
        'text': 'Cumulative Christian Martyrdoms (1st-5th Centuries)',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'color': '#1e293b'}
    },
    xaxis_title='Year (AD)',
    yaxis_title='Total Estimated Deaths',
    template='plotly_white',
    height=500,
    hovermode='x',
    font=dict(size=12),
    xaxis=dict(showgrid=True, gridcolor='#e2e8f0'),
    yaxis=dict(showgrid=True, gridcolor='#e2e8f0')
)

fig_cumulative.show()
fig_cumulative.write_html('martyrdoms_cumulative.html')
print("Chart saved as 'martyrdoms_cumulative.html'")

## Analysis by Emperor

In [ ]:
# Group by emperor
emperor_summary = df.groupby('Emperor').agg({
    'Deaths': 'sum',
    'Event': 'count'
}).rename(columns={'Event': 'Number of Events'}).sort_values('Deaths', ascending=False)

print("\nMartydoms by Emperor:")
print(emperor_summary)

# Visualization
fig_emperor = px.bar(
    emperor_summary.reset_index(),
    x='Emperor',
    y='Deaths',
    title='Total Recorded Martyrdoms by Emperor',
    labels={'Deaths': 'Estimated Number of Deaths', 'Emperor': 'Roman Emperor'},
    color='Deaths',
    color_continuous_scale='Reds',
    height=500
)

fig_emperor.update_layout(
    template='plotly_white',
    hovermode='x unified',
    xaxis_tickangle=-45
)

fig_emperor.show()
fig_emperor.write_html('martyrdoms_by_emperor.html')
print("Chart saved as 'martyrdoms_by_emperor.html'")

## Key Findings

### Notable Observations:

1. **Diocletian's Great Persecution (303 AD)** accounts for ~45% of all recorded early church martyrdoms
2. **Nero's Persecution (64-68 AD)** was the first major official persecution
3. **2nd-3rd Centuries**: Sporadic persecutions with relatively lower death tolls
4. **Constantine's Legalization (313 AD)** marked the end of systematic Roman persecution
5. **5th Century**: Persecution largely ceased in the Roman Empire but continued in some regions (e.g., Armenia)

### Data Limitations:

- Many estimates are based on historical records and church tradition
- Exact dates and death tolls for large persecutions are often uncertain
- Some figures (George, Catherine, Sebastian) have legendary elements mixed with historical accounts
- This represents only documented/recorded martyrdoms, likely undercounting actual totals

In [ ]:
# Export data to CSV for reference
df.to_csv('early_church_martyrdoms.csv', index=False)
print("Data exported to 'early_church_martyrdoms.csv'")